In [1]:
import tensorflow as tf

In [2]:
l0 = [1,2,3]
l1 = [4,5,6,7]
l2 = [7,8,9]
l3 = [1,2]

feat = [l0,l1,l2,l3]
label = [1,0,1,1]

In [109]:
flat_values = tf.concat(feat, axis=0)
row_lengths = [len(feature) for feature in feat]
ragged_tensor = tf.RaggedTensor.from_row_lengths(flat_values, row_lengths)

In [110]:
print(flat_values)

tf.Tensor([1 2 3 4 5 6 7 7 8 9 1 2], shape=(12,), dtype=int32)


In [111]:
features = tf.data.Dataset.from_tensor_slices(ragged_tensor)

In [112]:
for element in features:
    print(f"Feature is {element}")

Feature is [1 2 3]
Feature is [4 5 6 7]
Feature is [7 8 9]
Feature is [1 2]


In [124]:
# Define a function to batch ragged tensors
def ragged_batch_fn(batch):
    return tf.ragged.stack(batch)

In [125]:
# Apply custom batching
batched_dataset = features.batch(2).map(lambda batch: ragged_batch_fn(batch))

In [126]:
for batch in batched_dataset:
    print(batch)

<tf.RaggedTensor [[[1, 2, 3], [4, 5, 6, 7]]]>
<tf.RaggedTensor [[[7, 8, 9], [1, 2]]]>


In [30]:
a = tf.constant(['a','b'])
b = tf.constant(['c'])
c = tf.ragged.stack([a,b])
print(c)

<tf.RaggedTensor [[b'a', b'b'], [b'c']]>


In [36]:
r = tf.RaggedTensor.from_row_lengths(a,[2])
r

<tf.RaggedTensor [[b'a', b'b']]>

In [3]:
def dataset_generator(features, labels):
    def func():
        for i in range(len(features)):
            yield (features[i], labels[i])
    return func

In [4]:
dataset = tf.data.Dataset.from_generator(dataset_generator(feat, label),
                                         output_signature=(
                                             tf.TensorSpec(shape=(None,), dtype=tf.int32),
                                             tf.TensorSpec(shape=(), dtype=tf.int8)
                                         ))


In [26]:
# Define a function to batch ragged tensors
def ragged_batch_fn2(features, labels):
    return tf.RaggedTensor.from_row_lengths(features, [len(features)]), labels

In [27]:
ragged_batch_dataset = dataset.map(ragged_batch_fn2).batch(2)

In [28]:
for elements in ragged_batch_dataset:
    print(elements)

(<tf.RaggedTensor [[[1, 2, 3]], [[4, 5, 6, 7]]]>, <tf.Tensor: shape=(2,), dtype=int8, numpy=array([1, 0], dtype=int8)>)
(<tf.RaggedTensor [[[7, 8, 9]], [[1, 2]]]>, <tf.Tensor: shape=(2,), dtype=int8, numpy=array([1, 1], dtype=int8)>)


In [38]:
padded_batch_dataset = ragged_batch_dataset.map(lambda feature_batch, label_batch: (tf.squeeze(feature_batch.to_tensor(),axis=1), label_batch))

In [39]:
for elements in padded_batch_dataset:
    print(elements)

(<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
array([[1, 2, 3, 0],
       [4, 5, 6, 7]])>, <tf.Tensor: shape=(2,), dtype=int8, numpy=array([1, 0], dtype=int8)>)
(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[7, 8, 9],
       [1, 2, 0]])>, <tf.Tensor: shape=(2,), dtype=int8, numpy=array([1, 1], dtype=int8)>)


In [41]:
l = [[1,2,3],['a','b','c'],['aa','bb','cc']]
r = zip(*l)
t = list(r)
print(t)

[(1, 'a', 'aa'), (2, 'b', 'bb'), (3, 'c', 'cc')]


In [43]:
r2 = zip(*t)
list(r2)

[(1, 2, 3), ('a', 'b', 'c'), ('aa', 'bb', 'cc')]

In [1]:
import tensorflow as tf

# Define the TextVectorization layer with ngrams
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=10000,
    output_mode='int',
    ngrams=2  # Generate bigrams
)

# Sample text data
texts = ["I love coding", "TensorFlow is great"]

# Adapt the vectorizer to the texts (builds the vocabulary including n-grams)
vectorize_layer.adapt(texts)

# Apply the TextVectorization layer
vectorized_texts = vectorize_layer(texts)
print(vectorized_texts.numpy())

[[ 9  5 11  8  4]
 [ 3  7 10  2  6]]


In [2]:
vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'tensorflow is',
 'tensorflow',
 'love coding',
 'love',
 'is great',
 'is',
 'i love',
 'i',
 'great',
 'coding']

In [ ]:
tf.keras.optimizer.RMSprop